In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

# Building Project

## Parameters
### Dimension of the Building 

Each Room has a cubic shape

https://github.com/dm4bem-2023/1-model-team7/blob/c375fabf4c8fb06f8ce08dd8fe572f1d180b50a7/Schema.png 

In [2]:
l = 3
Sg = l**2
Sc = Si = 10 * Sg 

We consider that M1,M2,M3 are inside the building and M4 is outside. Then the only exchange are between the inside and the outside with M4 and between the two rooms with M5

### Thermo-Physical Properties
We consider than the wall is made of a width of insulation and a width of concrete (eM=eI+eC)

In [3]:
concrete = {'Conductivity': 1.400,
            'Density': 2400.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 9 * l**2}

insulation = {'Conductivity': 0.027,
              'Density': 55.0,
              'Specific heat': 1210,
              'Width': 0.1,
              'Surface': 9 * l**2}

glass = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 2*(l**2)}

wall = pd.DataFrame.from_dict({'Layer_out': insulation,
                               'Layer_in': concrete,
                               'Glass': glass},
                              orient='index')
wall

air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


### Radiative properties

In [4]:
# radiative properties
ε_wLW = 0.85    # long wave emmisivity: wall surface (concrete)
ε_gLW = 0.90    # long wave emmisivity: glass pyrex
α_wSW = 0.25    # short wave absortivity: white smooth surface
α_gSW = 0.38    # short wave absortivity: reflective blue glass
τ_gSW = 0.30    # short wave transmitance: reflective blue glass

σ = 5.67e-8     # W/(m²⋅K⁴) Stefan-Bolzmann constant
print(f'σ = {σ} W/(m²⋅K⁴)')

σ = 5.67e-08 W/(m²⋅K⁴)


### Convection properties

In [5]:
h0 = pd.DataFrame([{'in': 8., 'out': 25}], index=['h0'])  # W/(m²⋅K)
# h = pd.DataFrame([{'in': 8., 'out': 25}])  # W/(m²⋅K)
h0

,in,out
h0,8.0,25


To take into account the long wave radiation, we add 4 at the value of h_in

In [6]:
h = pd.DataFrame([{'in': 12., 'out': 25}], index=['h'])  # W/(m²⋅K)
# h = pd.DataFrame([{'in': 8., 'out': 25}])  # W/(m²⋅K)
h

,in,out
h,12.0,25


### Thermal circuit

Figure bellow shows the models of:

concrete & insulation wall: in red;
glass window: in green;
ventilation: in magenta;
indoor volume: in blue (conductances 6 & 7 for convection; conductance 5 for long wave radiation between the walls and the glass window);
HVAC system: in black.

Here is the thermal circuit :
https://github.com/dm4bem-2023/1-model-team7/blob/c375fabf4c8fb06f8ce08dd8fe572f1d180b50a7/Schema%20thermique.png  

The sources are:

 T0 outdoor temperature, °C;
 Ti,sp setpoint temperaure for the indoor air, °C;
 ϕ0 solar radiation absorbed by the outdoor surface of the wall, W;
 ϕi solar radiation absorbed by the indoor surface of the wall, W;
 Qa auxiliary heat gains (i.e., occupants, electrical devices, etc.), W;
 ϕa solar radiation absorbed by the glass, W.

In [7]:
# Conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns={'Conductance'})

,Conductance
Layer_out,21.87
Layer_in,567.00
Glass,630.00


In [8]:
# Convection wall
Gw = h * wall['Surface'][0]     # wall
print(f'Gw = {Gw} W/K')

Gw =       in   out
h  972.0  2025 W/K


In [9]:
# Convection glass
Gg = h * wall['Surface'][2]     # glass
print(f'Gg = {Gg} W/K')

Gg =       in  out
h  216.0  450 W/K


In [10]:
# Long wave radiation

# view factor wall-glass
Fwg = glass['Surface'] / concrete['Surface']
print(f'Fwg = {Fwg}')

Fwg = 0.2222222222222222


As explained before we do not need these factors because it is included in the value of h_in

In [11]:
# ventilation flow rate
Va = l**3                   # m³, volume of air
ACH = 1                     # air changes per hour
Va_dot = ACH / 3600 * Va    # m³/s, air infiltration
print(f'Va_dot = {Va_dot} m³/s')

Va_dot = 0.0075 m³/s


In [12]:
# ventilation & advection
Gv = air['Density'] * air['Specific heat'] * Va_dot
print(f'Gv = {Gv} W/K')

Gv = 9.0 W/K


In [13]:
# Proportionnal Controller
# P-controler gain
Kp = 1e6            # almost perfect controller Kp -> ∞
#Kp = 1e-3           # no controller Kp -> 0
#Kp = 0

## Thermal Capacities
### Walls

In [14]:
C = wall['Density'] * wall['Specific heat'] * wall['Surface'] * wall['Width']
C['Air'] = air['Density'] * air['Specific heat'] * Va
pd.DataFrame(C, columns={'Capacity'})

,Capacity
Layer_out,539055.0
Layer_in,34214400.0
Glass,2178000.0
Air,32400.0


## System of algebraic_differential equations (DAE)
### A : Incident Matrix

In [15]:
A = np.zeros([18, 11])       # n° of branches X n° of nodes
A[0, 0] = 1                 # branch 0: -> node 0
A[1, 0], A[1, 1] = -1, 1    # branch 1: node 0 -> node 1
A[2, 1], A[2, 2] = -1, 1    # branch 2: node 1 -> node 2
A[3, 2], A[3, 3] = -1, 1    # branch 3: node 2 -> node 3
A[4, 3], A[4, 4] = -1, 1    # branch 4: node 3 -> node 4
A[5, 4], A[5, 5] = -1, 1    # branch 5: node 4 -> node 5
A[6, 5], A[6, 6] = -1, 1    # branch 6: node 5 -> node 6
A[7, 6], A[7, 7] = -1, 1    # branch 7: node 6 -> node 7
A[8, 7], A[8, 8]= 1, -1     # branch 8: node 7 -> node 8
A[9, 8]= 1                 # branch 9: -> node 8
A[10, 7], A[10, 9]= -1, 1   # branch 10: node 7 -> node 9
A[11, 9], A[11, 6]= 1, -1   # branch 11: node 9 -> node 6
A[12, 10], A[12, 2]= -1, 1  # branch 12: node 10-> node 2
A[13, 1], A[13, 10]= -1, 1  # branch 13: node 1 -> node 10
A[14, 10] = 1               # branch 14: -> node 10
A[15, 10] = 1               # branch 15: -> node 10
A[16, 9] = 1                # branch 16: -> node 9
A[17, 9] = 1                # branch 17: -> node 9

# np.set_printoptions(suppress=False)
# pd.DataFrame(A)

### G : Incident Matrix

In [16]:
G = np.zeros([18, 18])       # n° of branches X n° of nodes
G[0, 0] = (h['out']*glass['Surface'])/2                
G[1, 1] = (glass['Conductivity']*glass['Surface']/2)/glass['Width']
G[2, 2] = 0
G[3, 3] = (concrete['Conductivity']*concrete['Surface']/2)/concrete['Width']
G[4, 4] = (concrete['Conductivity']*concrete['Surface']/2)/concrete['Width']              
G[5, 5] = (insulation['Conductivity']*insulation['Surface']/2)/insulation['Width']
G[6, 6] = (insulation['Conductivity']*insulation['Surface']/2)/insulation['Width']
G[7, 7] = 0
G[8, 8] = (glass['Conductivity']*glass['Surface']/2)/glass['Width']                  
G[9, 9] = (h['out']*glass['Surface'])/2 
G[10, 10] = (h['in']*glass['Surface'])/2  
G[11, 11] = (h['in']*concrete['Surface'])/2 
G[12, 12] = (h['in']*concrete['Surface'])/2                   
G[13, 13] = (h['in']*glass['Surface'])/2 
G[14, 14] = Gv 
G[15, 15] = Kp
G[16, 16] = Kp                  
G[17, 17] = Gv
print(f'G = ', G)

G =  [[2.2500e+02 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 3.1500e+02 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 2.8350e+02 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 2.8350e+02 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00
  0.0000e+00 0.0000e+00 0.0000e+00 0.00

### C : Capacity Matrix

In [17]:
neglect_air_glass = True

if neglect_air_glass:
    C = np.diag([0, 0, 0, C['Layer_out'], 0, C['Layer_in'], 0, 0,
                 0, 0, 0])
else:
    C = np.diag([C['Glass'], 0, 0, C['Layer_out'], 0, C['Layer_in'], 0, 0, C['Glass'],
                 C['Air'], C['Air']])

# pd.set_option("display.precision", 3)
# pd.DataFrame(C)

### b : temperature source vector 

In [18]:
b = np.zeros(18)        # branches
b[[0, 9, 14, 15, 16, 17]] = 1   # branches with temperature sources
print(f'b = ', b)

b =  [1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1.]


### f : heat flow source  vector 

In [19]:
f = np.zeros(11)         # nodes
f[[0, 2, 6, 8, 9, 10]] = 1     # nodes with heat-flow sources
print(f'f = ', f)

f =  [1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1.]


### y : output vector 

In [20]:
y = np.zeros(11)         # nodes
y[[9, 10]] = 1              # nodes (temperatures) of interest
print(f'y = ', y)

y =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.]


## State-space representation


In [21]:
[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
print('As = \n', As, '\n')
print('Bs = \n', Bs, '\n')
print('Cs = \n', Cs, '\n')
print('Ds = \n', Ds, '\n')

As = 
 [[-3.51632886e-04  1.95321179e-05]
 [ 3.07732587e-07 -6.20298705e-07]] 

Bs = 
 [[1.96748470e-08 0.00000000e+00 2.98870294e-09 3.32078105e-04
  0.00000000e+00 0.00000000e+00 8.74437643e-11 6.83666992e-07
  0.00000000e+00 0.00000000e+00 0.00000000e+00 3.32078105e-10]
 [0.00000000e+00 1.85175438e-11 0.00000000e+00 0.00000000e+00
  3.12544787e-07 2.81290308e-12 0.00000000e+00 0.00000000e+00
  6.43452704e-10 8.23001947e-14 3.12544787e-13 0.00000000e+00]] 

Cs = 
 [[0.00000000e+00 1.06935324e-05]
 [1.79008363e-04 0.00000000e+00]] 

Ds = 
 [[0.00000000e+00 5.92429721e-05 0.00000000e+00 0.00000000e+00
  9.99921064e-01 8.99928958e-06 0.00000000e+00 0.00000000e+00
  9.77917911e-07 2.63302098e-07 9.99921064e-07 0.00000000e+00]
 [5.92330006e-05 0.00000000e+00 8.99777485e-06 9.99752761e-01
  0.00000000e+00 0.00000000e+00 2.63257780e-07 6.31422796e-07
  0.00000000e+00 0.00000000e+00 0.00000000e+00 9.99752761e-07]] 



### Steady State

First we are going to check the model in steady state to see if it is correct.
We are going to consider that : 

- The controller is not active, $K_p \rightarrow 0$,
- The outdoor temperature is $T_o = 10 \, \mathrm{^\circ C}$,
- The indoor temperature setpoint is $T_{i,sp} = 20 \, \mathrm{^\circ C}$,
- All flow rate sources are zero.

In [22]:
b = np.zeros(18)        # temperature sources
b[[0, 9, 14, 17]] = 10      # outdoor temperature
b[[15, 16]] = 20            # indoor set-point temperature
print(f'b = ', b)

f = np.zeros(11)         # flow-rate sources
print(f'f = ', f)

b =  [10.  0.  0.  0.  0.  0.  0.  0.  0. 10.  0.  0.  0.  0. 10. 20. 20. 10.]
f =  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


### System of Differential Algebraic Equations (DAE)
The value of temperature in steady-state is obtained from the system of DAE by considering that $C \dot{\theta} = 0$ , then we have : $$\theta_{ss} = (A^T G A)^{-1}(A^T G b + f)$$

In [23]:
θ = np.linalg.inv(A.T @ G @ A) @ (A.T @ G @ b + f)
print(f'θ = {θ} °C')

θ = [12.63304914 14.51379853 19.99931757 19.99931757 19.99931757 19.99931757
 19.99931757 14.51379853 12.63304914 19.99931757 19.99931757] °C


In [24]:
[As, Bs, Cs, Ds] = dm4bem.tc2ss(A, G, b, C, f, y)
print('As = \n', As, '\n')
print('Bs = \n', Bs, '\n')
print('Cs = \n', Cs, '\n')
print('Ds = \n', Ds, '\n')

As = 
 [[-3.51632886e-04  1.95321179e-05]
 [ 3.07732587e-07 -6.20298705e-07]] 

Bs = 
 [[1.96748470e-08 0.00000000e+00 2.98870294e-09 3.32078105e-04
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.85175438e-11 0.00000000e+00 0.00000000e+00
  3.12544787e-07 2.81290308e-12]] 

Cs = 
 [[0.00000000e+00 1.06935324e-05]
 [1.79008363e-04 0.00000000e+00]] 

Ds = 
 [[0.00000000e+00 5.92429721e-05 0.00000000e+00 0.00000000e+00
  9.99921064e-01 8.99928958e-06]
 [5.92330006e-05 0.00000000e+00 8.99777485e-06 9.99752761e-01
  0.00000000e+00 0.00000000e+00]] 



### State-space representation
The input vector u is described bellow : 
$$
u = \begin{bmatrix}
b_T \\
f_Q
\end{bmatrix}
$$

where:
- $b_T$ is a vector of the nonzero elements of vector $b$ of temperature sources. 
    - $b_T = [T_o, T_o, T_o, T_{i,sp}, T_{i,sp},T_o]^T$ corresponding to branches 0, 9, 14, 15, 16 and 17, 

- $f_Q$ vector: the nonzero elements of vector $f$ of flow sources. $f_Q = [\Phi_a, \Phi_i, \Phi_i, \Phi_a, \dot{Q}_a, \dot{Q}_a]^T$ corresponding to nodes 0, 2, 6, 8, 9, 10 where:
    - $\Phi_o$: solar radiation absorbed by the outdoor surface of the wall, W;
    - $\Phi_i$: solar radiation absorbed by the indoor surface of the wall, W;
    - $\dot{Q}_a$: auxiliary heat gains (i.e., occupants, electrical devices, etc.), W;
    - $\Phi_a$: solar radiation absorbed by the glass, W.

In [25]:
bT = np.array([[10], [10], [10], [20], [20], [10]])     # [To, To, To, Tisp, Tisp, To]
fQ = np.array([[0], [0], [0], [0], [0], [0]])         # [Φa, Φi, Φi, Φa, Qa, Qa]
u = np.hstack([bT, fQ])
print(f'u = {u}')

u = [[10  0]
 [10  0]
 [10  0]
 [20  0]
 [20  0]
 [10  0]]


In [26]:
yss = (-Cs @ np.linalg.inv(As) @ Bs + Ds) @ u
print(f'yss = {yss} °C')

yss = [[19.99931757  0.        ]
 [19.99931757  0.        ]] °C


In [27]:
eps1=(abs(θ[9] - yss[0,0]))
print(f'eps1 = \
{eps1:.2e} °C')
eps2=(abs(θ[10] - yss[1,0]))
print(f'eps2 = \
{eps2:.2e} °C')
print(f'Max error between DAE and state-space: \
{max(eps1,eps2):.2e} °C')

eps1 = 3.55e-15 °C
eps2 = 3.55e-15 °C
Max error between DAE and state-space: 3.55e-15 °C


## Dynamic Simulation

### Time Step
To set up the time step we are going to determine the maximum time step : 

In [28]:
λ = np.linalg.eigvals(As)    # eigenvalues of matrix As

print('Time constants : \n', -1 / λ, 's \n')
print('2 x Time constants : \n', -2 / λ, 's \n')
dtmax = 2 * min(-1. / λ)
print(f'Maximum time step: {dtmax:.6f} s = {dtmax / 60:.6f} min')

Time constants : 
 [   2843.73659554 1657891.61112565] s 

2 x Time constants : 
 [   5687.47319108 3315783.22225129] s 

Maximum time step: 5687.473191 s = 94.791220 min


We will chose a time step smaller than : $\Delta t_{\text{max}} = \min \left(-\frac{2}{\lambda_i}\right)$

In [29]:
# time step

dt = np.floor(dtmax / 60) * 60   # s
print(f'dt = {dt} s = {dt / 60:.0f} min')

dt = 5640.0 s = 94 min


pour avoir time step pas trop court approximation de négligence capacité de l'air et de la vitre 

### Settling time
We consider that the settling time is roughly 4 times the larger time constant.

In [30]:
# settling time
time_const = np.array([int(x) for x in sorted(-1 / λ)])
print('4 * Time constants: \n', 4 * time_const, 's \n')

t_settle = 4 * max(-1 / λ)
print(f'Settling time: \
{t_settle:.0f} s = \
{t_settle / 60:.1f} min = \
{t_settle / (3600):.2f} h = \
{t_settle / (3600 * 24):.2f} days')

4 * Time constants: 
 [  11372 6631564] s 

Settling time: 6631566 s = 110526.1 min = 1842.10 h = 76.75 days


### Step response
We are now going to determine the dynamic response of the system to a step input.

#### Duration
We pay attention to make the duration of the simulation larger than the estimated settling time. Then it will give a number of time steps in the time vector

In [31]:
# Step response
# -------------
# Find the next multiple of 3600 s that is larger than t_settle
duration = np.ceil(t_settle / 3600) * 3600
n = int(np.floor(duration / dt))    # number of time steps
t = np.arange(0, n * dt, dt)        # time vector for n time steps

print(f'Duration = {duration} s')
print(f'Number of time steps = {n}')
# pd.DataFrame(t, columns=['time'])

Duration = 6634800.0 s
Number of time steps = 1176


#### Input vector
The input vector as the same shape as in the last part : 
$$
u = \begin{bmatrix}
b_T \\
f_Q
\end{bmatrix}
$$


In [32]:
u = np.zeros([12, n])                # u = [To To To Tisp Tisp To Φa, Φi, Φi, Φa, Qa, Qa]
u[0:3, :] = 10 * np.ones([3, n])    # To = 10 for n time steps pr (0, 1, 2)
u[3:5, :] = 20 * np.ones([2, n])      # Tisp = 20 for n time steps pr (3, 4)
u[5, :] = 10 * np.ones([1, n])      # To = 10 for n time steps pr (5)

# pd.DataFrame(u)

#### Time Integration

In [33]:
n_s = As.shape[0]                      # number of state variables
θ_exp = np.zeros([n_s, t.shape[0]])    # explicit Euler in time t
θ_imp = np.zeros([n_s, t.shape[0]])    # implicit Euler in time t

I = np.eye(n_s)                        # identity matrix

for k in range(n - 1):
    θ_exp[:, k + 1] = (I + dt * As) @\
        θ_exp[:, k] + dt * Bs @ u[:, k]
    θ_imp[:, k + 1] = np.linalg.inv(I - dt * As) @\
        (θ_imp[:, k] + dt * Bs @ u[:, k])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 12 is different from 6)

we obtain

In [ ]:
y_exp = Cs @ θ_exp + Ds @  u
y_imp = Cs @ θ_imp + Ds @  u

The results of explicit and implicit Euler integration are practically identical.

In [ ]:
fig, ax = plt.subplots()
ax.plot(t / 3600, y_exp.T, t / 3600, y_imp.T)
ax.set(xlabel='Time, $t$ / h',
       ylabel='Temperatue, $θ_i$ / °C',
       title='Step input: outdoor temperature $T_o$')
ax.legend(['Explicit', 'Implicit'])
ax.grid()
plt.show()